In [ ]:
# %load myui_v5_Async.py

# Form implementation generated from reading ui file 'myui_v5.ui'
#
# Created by: PyQt5 UI code generator 5.15.2
#
# WARNING: Any manual changes made to this file will be lost when pyuic5 is
# run again.  Do not edit this file unless you know what you are doing.
import sqlite3

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import *
import re

from bs4 import BeautifulSoup
import requests

import asyncio
from aiohttp import ClientSession


class Ui_MainWindow(object):
    #
    # def __init__(self,message):
    #
    #     print('message from '+message)

    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(770, 600)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")

        self.groupBox = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox.setGeometry(QtCore.QRect(0, 20, 781, 281))
        self.groupBox.setObjectName("groupBox")

        self.calendarWidget = QtWidgets.QCalendarWidget(self.groupBox)
        self.calendarWidget.setGeometry(QtCore.QRect(10, 50, 351, 181))
        self.calendarWidget.setObjectName("calendarWidget")

        self.lineEdit = QtWidgets.QLineEdit(self.groupBox)
        self.lineEdit.setGeometry(QtCore.QRect(400, 170, 311, 51))
        font = QtGui.QFont()
        font.setPointSize(14)
        self.lineEdit.setFont(font)
        self.lineEdit.setObjectName("lineEdit")

        self.pushButton = QtWidgets.QPushButton(self.groupBox)
        self.pushButton.setGeometry(QtCore.QRect(310, 240, 131, 32))
        self.pushButton.setObjectName("pushButton")

        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setGeometry(QtCore.QRect(310, 310, 131, 32))
        self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        self.label_3.setWordWrap(False)
        self.label_3.setObjectName("label")

        self.pushButton2 = QtWidgets.QPushButton(self.groupBox)
        self.pushButton2.setGeometry(QtCore.QRect(500, 80, 131, 32))
        self.pushButton2.setObjectName("pushButton2")

        self.label = QtWidgets.QLabel(self.groupBox)
        self.label.setGeometry(QtCore.QRect(60, 30, 241, 16))
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setWordWrap(False)
        self.label.setObjectName("label")

        self.label_2 = QtWidgets.QLabel(self.groupBox)
        self.label_2.setGeometry(QtCore.QRect(500, 140, 101, 16))
        self.label_2.setScaledContents(False)
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setObjectName("label_2")

        self.scrollArea = QtWidgets.QScrollArea(self.centralwidget)
        self.scrollArea.setGeometry(QtCore.QRect(35, 340, 700, 230))  # left, top, width, height
        # self.scrollArea.setMinimumSize(QtCore.QSize(0, 1000))
        self.scrollArea.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOn)
        self.scrollArea.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOn)
        self.scrollArea.setWidgetResizable(True)
        self.scrollArea.setObjectName("scrollArea")

        self.scrollAreaWidgetContents = QtWidgets.QWidget()
        # self.scrollAreaWidgetContents.setGeometry(QtCore.QRect(65, 350, 700, 300))
        self.scrollAreaWidgetContents.setObjectName("scrollAreaWidgetContents")
        self.scrollArea.setWidget(self.scrollAreaWidgetContents)

        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 770, 24))
        self.menubar.setObjectName("menubar")
        self.menu = QtWidgets.QMenu(self.menubar)
        self.menu.setObjectName("menu")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.menubar.addAction(self.menu.menuAction())

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        self.Vbox = QtWidgets.QVBoxLayout(self.scrollAreaWidgetContents)
        # self.groupBox = QtWidgets.QGroupBox(self.groupBox)
        # self.groupBox.setGeometry(QtCore.QRect(400, 50, 300, 80))
        # self.groupBox.setObjectName("groupBox2")

    # def openWindow(self):
    #     from fav_window_v2 import Ui_FavWindow2
    #
    #     self.window = QtWidgets.QMainWindow()
    #     self.ui = Ui_FavWindow2()
    #     self.ui.setupUi(self.window)
    #     self.window.show()
    #     self.close()

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.groupBox.setTitle(_translate("MainWindow", "GroupBox"))
        self.lineEdit.setPlaceholderText(_translate("MainWindow", "輸入任何關鍵字"))
        # self.lineEdit.setText(_translate('MainWindow',self.))
        self.pushButton.setText(_translate("MainWindow", "搜尋"))
        self.label.setText(_translate("MainWindow", "日期搜尋（預設為自選日期至今日）"))
        self.label_2.setText(_translate("MainWindow", "關鍵字搜尋"))
        self.menu.setTitle(_translate("MainWindow", "日本綜藝節目搜尋程式"))
        self.label_3.setText(_translate("MainWindow", "搜尋結果"))

        self.pushButton2.setText(_translate("MainWindow", "已收藏"))


class MyMainWindow(QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):
        super(MyMainWindow, self).__init__(parent)
        self.setupUi(self)
        self.listWidget = QtWidgets.QListWidget()
        self.pushButton.clicked.connect(self.showContent)
        self.pushButton2.clicked.connect(self.open_window2)
        self.w = None
        # print(message)



    async def get_chosenDate_soup(self, date_num, keyword, session):

        page = 1
        title_list = []
        link_list = []

        while True:
            base_url = 'https://jshow.tv/more-post/page/' + str(page)
            print("page" + str(page))
            async with session.get(base_url) as response:
                html_body = await response.text()
            # response = requests.get(base_url)
                soup = BeautifulSoup(html_body, 'lxml')
                div_titles = soup.find_all('div', class_='des')
                video_date_num = int()
                date_pattern = re.compile(r'\d\d\d\d\d\d')
                try:
                    for div_title in div_titles:
                        video_date_num = date_pattern.search(div_title.getText()).group(0)
                        if keyword in div_title.getText():
                            for div_child in div_title.children:
                                if date_pattern.search(div_child.getText()[-6:]) is not None:
                                    title_list.append(div_title.getText())
                                    link_list.append(div_title.find_previous_sibling('div').findChild('a')['href'])
                                    break
                                else:
                                    print('date pattern not pass!!')

                except Exception as e:
                    print(e)

            if int(video_date_num) >= int(date_num):
                page += 1
                # print('page: ' + str(page))
            else:
                break

        return title_list, link_list

    async def showContent(self):
        self.clearLayout(self.Vbox)
        datenum = self.calendarWidget.selectedDate().toString("yyMMdd")
        keywords = self.lineEdit.text()
        keywords = keywords.split('、')


        for keyword in keywords:
            async with ClientSession() as session:
                tasks = [asyncio.create_task(self.get_chosenDate_soup(int(datenum), keyword, session))]
                await asyncio.create_task(tasks)
            title_list, link_list = self.get_chosenDate_soup(int(datenum), keyword)
            # linkTemplate = '<a href={0}>{1}</a>'
            try:
                for title, link in zip(title_list, link_list):

                    video_title = str(title).split('内容：')[0][0:-6]
                    video_cast = '出演： ' + str(title).split('出演：')[1]
                    video_content = '內容： ' + str(title).split('出演：')[0].split('内容：')[1]
                    video_date = str(title).split('内容：')[0][-6:]

                    date_label = QLabel(video_date)
                    title_label = QLabel('<a href = ' + link + '>' + video_title + '</a>')
                    cast_label = QLabel(video_cast)
                    content_label = QLabel(video_content)
                    title_label.setOpenExternalLinks(True)

                    video_widgets = [title_label, cast_label, content_label]
                    for video_widget in video_widgets:
                        self.Vbox.addWidget(video_widget)
            except:
                print('somethings wrong')

    def clearLayout(self, layout):
        if layout is not None:
            while layout.count():
                item = layout.takeAt(0)
                widget = item.widget()
                if widget is not None:
                    widget.deleteLater()
                else:
                    self.clearLayout(item.layout())

    def open_window2(self, checked):
        if self.w is None:
            self.w = SecondWindow()
        self.w.show()

        # self.close()


dbfile = "J_VarietyShow.db"


class SecondWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.w = None
        _translate = QtCore.QCoreApplication.translate
        layout = QGridLayout()
        self.label = QLabel("Another Window")
        layout.addWidget(self.label)

        self.lineEdit = QLineEdit()
        self.lineEdit.setGeometry(QtCore.QRect(70, 310, 271, 30))
        self.lineEdit.setObjectName("lineEdit")
        layout.addWidget(self.lineEdit)

        self.pushButton = QPushButton()
        self.pushButton.setGeometry(QtCore.QRect(360, 310, 82, 25))
        self.pushButton.setObjectName("pushButton")
        self.pushButton.setText(_translate("MainWindow", "新增收藏"))
        self.pushButton.clicked.connect(self.add_listItem)
        layout.addWidget(self.pushButton)

        self.label = QLabel()
        self.label.setGeometry(QtCore.QRect(145, 20, 211, 16))
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setObjectName("label")
        self.label.setText(_translate("MainWindow", "我的收藏關鍵字"))
        layout.addWidget(self.label)

        self.line = QFrame()
        self.line.setGeometry(QtCore.QRect(10, 350, 491, 16))
        self.line.setFrameShape(QtWidgets.QFrame.HLine)
        self.line.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.line.setObjectName("line")
        layout.addWidget(self.line)

        self.tableWidget = QTableWidget()
        self.tableWidget.setGeometry(QtCore.QRect(20, 40, 461, 261))
        self.tableWidget.setAlternatingRowColors(False)
        self.tableWidget.setColumnCount(2)
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setRowCount(0)
        self.tableWidget.horizontalHeader().setCascadingSectionResizes(False)
        self.tableWidget.setHorizontalHeaderLabels(['勾選', '關鍵字'])
        self.tableWidget.setSortingEnabled(True)
        layout.addWidget(self.tableWidget)

        self.pushButton_3 = QPushButton()
        self.pushButton_3.setGeometry(QtCore.QRect(120, 370, 261, 25))
        self.pushButton_3.setText(_translate("MainWindow", "刪除勾選"))
        self.pushButton_3.clicked.connect(self.delete_listItem)
        layout.addWidget(self.pushButton_3)

        self.pushButton_2 = QPushButton()
        self.pushButton_2.setGeometry(QtCore.QRect(120, 370, 261, 25))
        self.pushButton_2.setObjectName("pushButton_2")
        self.pushButton_2.setText(_translate("MainWindow", "我選好了!"))
        self.pushButton_2.clicked.connect(self.checked_search)
        layout.addWidget(self.pushButton_2)

        self.chkBoxItem = QtWidgets.QTableWidgetItem()

        self.setLayout(layout)

        conn = sqlite3.connect(dbfile)
        datas = conn.execute("select Keyword from Fav;")

        for row_num, row_data in enumerate(datas):
            self.tableWidget.insertRow(row_num)
            for column_num, data in enumerate(row_data):
                self.chkBoxItem = QtWidgets.QTableWidgetItem()
                self.chkBoxItem.setCheckState(QtCore.Qt.Unchecked)
                self.tableWidget.setItem(row_num, column_num, self.chkBoxItem)
                self.tableWidget.setItem(row_num, column_num + 1, QtWidgets.QTableWidgetItem(str(data)))

        conn.close()

    def delete_listItem(self):
        nrows = self.tableWidget.rowCount()
        del_rows = []
        for row in range(0, nrows):
            chk = self.tableWidget.item(row, 0)
            if chk.checkState() != 0 and nrows > 0:
                del_row = self.tableWidget.item(row, 1).row()
                del_rows.append(del_row)
                # self.tableWidget.removeRow(row-1)
                # print('已刪除：' + del_content.text())
        # 實際運行會發現，只有第一列正確被刪除
        # 錯誤的原因是因為第一列被刪除後，該列後方的所有位置也跟著減-1了！
        # 也就是當你只是單純地使用selectedIndexes()取得多選的列來刪除的話，從第2筆開始的指到row，已經不在是當初user選到要刪除的row了！

        del_rows = sorted(set(del_rows), reverse=True)  # 從後面開始刪除就行了！

        conn = sqlite3.connect(dbfile)
        for row in del_rows:
            del_content = self.tableWidget.item(row, 1)
            del_content = del_content.text()
            self.tableWidget.removeRow(row)
            print('已刪除：' + del_content)

            sql_str = "Delete from Fav where Keyword = ('{}');".format(del_content)
            print(sql_str)
            conn.execute(sql_str)
            conn.commit()

        conn.close()

    def add_listItem(self):
        addinput = self.lineEdit.text()
        conn = sqlite3.connect(dbfile)
        datas = conn.execute("select Keyword from Fav;")
        sql_str = "Insert into Fav(Keyword) values('{}');".format(addinput)
        conn.execute(sql_str)
        conn.commit()
        print("已新增: " + addinput)

        self.tableWidget.setRowCount(0)

        for row_num, row_data in enumerate(datas):  # 自動建立索引
            self.tableWidget.insertRow(row_num)
            for column_num, data in enumerate(row_data):
                self.chkBoxItem = QtWidgets.QTableWidgetItem()
                self.chkBoxItem.setCheckState(QtCore.Qt.Unchecked)
                self.tableWidget.setItem(row_num, column_num, self.chkBoxItem)
                self.tableWidget.setItem(row_num, column_num + 1, QtWidgets.QTableWidgetItem(str(data)))

        conn.close()
        # QApplication.processEvents()

    def checked_search(self):
        pass_list = []
        for row in range(0, self.tableWidget.rowCount()):
            chkstateitem = self.tableWidget.item(row, 0)
            item = self.tableWidget.item(row, 1)
            if chkstateitem.checkState() != 0:
                pass_list.append(item.text())
                # MyMainWindow.passedKeyword()

        # mainWindow = MyMainWindow()
        search = ''
        for s in pass_list:
            search = search + s + '、'
            # print('pass：'+ s)

        if self.w is None:
            self.w = MyMainWindow()
        self.w.lineEdit.setText(search[:-1])
        print(search)
        self.w.show()

        self.close()


if __name__ == "__main__":
    import sys

    app = QApplication(sys.argv)

    myWin = MyMainWindow()
    myWin.show()
    # sub_window()
    # sys.exit(app.exec_())
    app.exec_()
